#### Задание

1) Собрать информацию о вакансиях на вводимую должность с сайтa [hh.ru](https://hh.ru/?customDomain=1). Приложение должно анализировать несколько страниц сайта.
Получившийся список должен содержать в себе:
- Наименование вакансии
- Предлагаемую зарплату (дополнительно: разносим в три поля: минимальная, максимальная, валюта. Цифры преобразуем к цифрам)
- Ссылку на саму вакансию
- Сайт, откуда собрана вакансия

2) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

#### Решение

### 1.

In [30]:
# импортируем необходимые библиотеки

import pandas as pd
import requests
import json
from bs4 import BeautifulSoup as bs

In [31]:
# задаем headers
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
}

In [32]:
# создаем наш итоговый список с вакансиями    
vacancy_list = []

# проходим циклом по 10 страницам
for i in range(10):
    
    # Задаем адреса страниц - у нулевой страницы адрес отличается
    if i == 0:
        url = 'https://hh.ru/vacancies/data-scientist?customDomain=1'
    else:
        url = f'https://hh.ru/vacancies/data-scientist?customDomain=1&page={i}&hhtmFrom=vacancy_search_catalog'

    # парсим страницу целиком
    response = requests.get(url, headers = headers)
    dom = bs(response.text, 'html.parser')

    # создаем список тегов <div> у которых class = 'vacancy-serp-item__layout', что соответсвует блоку одной вакансии (посмотрели в браузере)
    tag_div_list = dom.find_all('div', {'class' : ['vacancy-serp-item__layout']})

    # проходим циклом по нашему списку вакансий со страницы
    for tag_div in tag_div_list:

        # задаем словарь с информауией о вакансии
        tag_div_dict = {}

        # находим тег с вакансией и ссылкой
        find_vacancy = tag_div.find('a', {'class' : 'bloko-link'})

        # находим значение зарплаты, через try/except, так как не всегда ЗП указана, в случае ошибки, присваиваем значение 'ЗП не указана'
        try:
            salary_vacancy = tag_div.find('span', {'data-qa' : 'vacancy-serp__vacancy-compensation'}).text.replace('\u202f', ' ')
        except AttributeError:
            salary_vacancy = 'ЗП не указана'

        # находим из find_vacation ссылку на вакансию
        href = find_vacancy['href']

        # создаем словарь где ключ - название вакансии, а значения - это ЗП и ссылка на вакансию.
        tag_div_dict[find_vacancy.text] = {'salary': salary_vacancy,'link': href, 'site': 'hh.ru'}

        # добавляем в наш итоговый список словарь с информацией о вакансии
        vacancy_list.append(tag_div_dict)


In [33]:
vacancy_list

[{'Data Scientist (команда Поиска)': {'salary': 'от 350 000 ₽',
   'link': 'https://hh.ru/vacancy/90547943?query=data+scientist&hhtmFrom=vacancy_search_list',
   'site': 'hh.ru'}},
 {'Data Scientist в направление "Работодатели"': {'salary': 'ЗП не указана',
   'link': 'https://hh.ru/vacancy/90547878?query=data+scientist&hhtmFrom=vacancy_search_list',
   'site': 'hh.ru'}},
 {'Data scientist / Аналитик данных HR-проектов': {'salary': 'ЗП не указана',
   'link': 'https://hh.ru/vacancy/92357631?query=data+scientist&hhtmFrom=vacancy_search_list',
   'site': 'hh.ru'}},
 {'Data scientist': {'salary': 'от 150 000 ₽',
   'link': 'https://hh.ru/vacancy/91945134?query=data+scientist&hhtmFrom=vacancy_search_list',
   'site': 'hh.ru'}},
 {'Junior Data Scientist в Блок Финансы (LLM)': {'salary': 'ЗП не указана',
   'link': 'https://hh.ru/vacancy/92401159?query=data+scientist&hhtmFrom=vacancy_search_list',
   'site': 'hh.ru'}},
 {'Аналитик / Data scientist': {'salary': 'ЗП не указана',
   'link': 'ht

In [34]:
# Если хотим преобразовать в DataFrame, то изменим структуру в словарь

# создаем словарь, нужной структуры
dict_for_DF = {'vacancy' : [], 'salary' : [], 'link' : [], 'site': []}

# пробегаемся циклом по каждой вакансии из цикла
for vacancy in vacancy_list:
    
    # добавляем в список по ключу 'vacancy' значение - название вакансии
    dict_for_DF['vacancy'].append(*list(vacancy.keys()))
    
    # добавляем в списки по остальным ключам значения, исключая уже первый ключ 'vacancy'
    for col in list(dict_for_DF.keys())[1:]:
        dict_for_DF[col].append(list(vacancy.values())[0][col])

dict_for_DF

{'vacancy': ['Data Scientist (команда Поиска)',
  'Data Scientist в направление "Работодатели"',
  'Data scientist / Аналитик данных HR-проектов',
  'Data scientist',
  'Junior Data Scientist в Блок Финансы (LLM)',
  'Аналитик / Data scientist',
  'Data Scientist (DataLab)',
  'Data Scientist',
  'Data Scientist',
  'Data Scientist (DataProd)',
  'Data Scientist / Аналитик Data Science',
  'Data Scientist (Middle)',
  'Data Scientist / ML engineer',
  'Data scientist',
  'Аналитик скоринга / Data Scientist',
  'Data Scientist (ОСАГО)',
  'Data analyst/Data Scientist',
  'Стажер Data scientist',
  'Data scientist NLP',
  'Data scientist (Персонализация)',
  'Data Scientist middle+/Senior',
  'Data Scientist',
  'Специалист (Data Scientist)',
  'Data Scientist, LLM & Generative AI',
  'Data Scientist',
  'Data Scientist',
  'Data Scientist (NLP)',
  'Data Scientist (LLM)',
  'Data engineer / Data Scientist / Data Analyst',
  'Data scientist',
  'Data Scientist (специалист по исследованию

In [35]:
# теперь преобразуем в DataFrame

df = pd.DataFrame(dict_for_DF)
df.head(10)

,vacancy,salary,link,site
0,Data Scientist (команда Поиска),от 350 000 ₽,https://hh.ru/vacancy/90547943?query=data+scie...,hh.ru
1,"Data Scientist в направление ""Работодатели""",ЗП не указана,https://hh.ru/vacancy/90547878?query=data+scie...,hh.ru
2,Data scientist / Аналитик данных HR-проектов,ЗП не указана,https://hh.ru/vacancy/92357631?query=data+scie...,hh.ru
3,Data scientist,от 150 000 ₽,https://hh.ru/vacancy/91945134?query=data+scie...,hh.ru
4,Junior Data Scientist в Блок Финансы (LLM),ЗП не указана,https://hh.ru/vacancy/92401159?query=data+scie...,hh.ru
5,Аналитик / Data scientist,ЗП не указана,https://hh.ru/vacancy/91834791?query=data+scie...,hh.ru
6,Data Scientist (DataLab),130 000 – 200 000 ₽,https://hh.ru/vacancy/91000409?query=data+scie...,hh.ru
7,Data Scientist,ЗП не указана,https://hh.ru/vacancy/92126316?query=data+scie...,hh.ru
8,Data Scientist,ЗП не указана,https://hh.ru/vacancy/91789730?query=data+scie...,hh.ru
9,Data Scientist (DataProd),150 000 – 230 000 ₽,https://hh.ru/vacancy/91000315?query=data+scie...,hh.ru


In [36]:
# сохраним vacancy_list в файл *.json

with open('output.json', 'w', encoding = 'utf-8') as f:
    json.dump(vacancy_list, f)

In [37]:
# также сохраним DataFrame в файл *.csv

df.to_csv('output.csv', encoding='utf-8-sig')

### 2.

In [38]:
from pprint import pprint

from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError 

In [39]:
# подключаемся к клиенту Mongo на локальной машине
client = MongoClient()

client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [40]:
# cоздание датабазы vacancies

db = client['vacancies']
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'vacancies')

In [41]:
# импортируем из json файла список вакансий, которые получили на прошлом задании и сохранили в файл

with open('output.json', 'r', encoding = 'utf-8') as f:
    vacancy_list = json.load(f)
    
# вывод для примера 3 первых вакансий
vacancy_list[:3]

[{'Data Scientist (команда Поиска)': {'salary': 'от 350 000 ₽',
   'link': 'https://hh.ru/vacancy/90547943?query=data+scientist&hhtmFrom=vacancy_search_list',
   'site': 'hh.ru'}},
 {'Data Scientist в направление "Работодатели"': {'salary': 'ЗП не указана',
   'link': 'https://hh.ru/vacancy/90547878?query=data+scientist&hhtmFrom=vacancy_search_list',
   'site': 'hh.ru'}},
 {'Data scientist / Аналитик данных HR-проектов': {'salary': 'ЗП не указана',
   'link': 'https://hh.ru/vacancy/92357631?query=data+scientist&hhtmFrom=vacancy_search_list',
   'site': 'hh.ru'}}]

In [42]:
# создадим коллекию vacancies в нашей базе данных vacancies и сохраним в переменную vacancies_collection 

vacancies_collection = db.vacancies
vacancies_collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'vacancies'), 'vacancies')

In [43]:
# создадим функцию добавления вакансий в нашу коллекцию вакансий в базе данных

def add_vacancies_in_vacancies_collection(vacancy_list):
    
    # уникальным идентификатором вакансии будем считать ссылку (название вакансии не всегда оригинальное)
    # создадим список ссылок наших вакансий в коллекции
    list_links = []
    for doc in vacancies_collection.find():
        list_links.append(doc['link'])
    
    # проходим циклом по каждой вакансии и формируем документ для записи в коллекцию
    for vacancy in vacancy_list:
       
        # название вакансии
        title = list(vacancy.keys())[0]
        
        # создаем документ с ключом 'title' и значением - название вакансии
        document = {'title': title}
     
        # проходим циклом по остальным атрибутам вакансии и записываем их значения в соответсвующую графу
        for key in vacancy[title].keys():
            
            document[key] = vacancy[title][key]
        
        # обновляем наш документ в коллекции, если ссылка от него в списке ссылок наших вакансий list_links
        # иначе добавляем новый документ в коллекцию
        if document['link'] in list_links:
            vacancies_collection.replace_one({'link': document['link']}, document)
        else:
            vacancies_collection.insert_one(document)

In [44]:
# и запускаем функцию для добавления данных в коллекцию

add_vacancies_in_vacancies_collection(vacancy_list)

In [45]:
# количество записей коллекции
# если запустить функцию еще раз, то данные в коллекцие просто перезапишутся, так как ссылки на вакансии те же

len(list(vacancies_collection.find()))

349

In [46]:
# выведем первые 3 вакансии из коллекции
list(vacancies_collection.find()[:3])

[{'_id': ObjectId('65b9ded25a713219e8fba556'),
  'title': 'Data Scientist (команда Поиска)',
  'salary': 'от 350 000 ₽',
  'link': 'https://hh.ru/vacancy/90547943?query=data+scientist&hhtmFrom=vacancy_search_list',
  'site': 'hh.ru'},
 {'_id': ObjectId('65b9ded25a713219e8fba557'),
  'title': 'Data Scientist в направление "Работодатели"',
  'salary': 'ЗП не указана',
  'link': 'https://hh.ru/vacancy/90547878?query=data+scientist&hhtmFrom=vacancy_search_list',
  'site': 'hh.ru'},
 {'_id': ObjectId('65b9ded25a713219e8fba558'),
  'title': 'Data scientist / Аналитик данных HR-проектов',
  'salary': 'ЗП не указана',
  'link': 'https://hh.ru/vacancy/92357631?query=data+scientist&hhtmFrom=vacancy_search_list',
  'site': 'hh.ru'}]

In [47]:
#vacancies_collection.delete_many({})